Dataset:

Het is een audiodataset. Een audio dataset heeft 3 dimensies. In pytorch is dat de 1d dimensie.

10 klassen:
13 features

8800 observaties
10 getallen
10 herhalingen
88 sprekers (88x10x10)

1 regel getallen is 1 uitspraak van 1 persoon
1 blok is 1 persoon die 1 uitspraak doet (getal 3 bijv)

Er wordt ook een onderscheid gemaakt tussen man en vrouw: Er zijn dus 10 getallen, gesplitst tussen man en vrouw, 20 klassen!

Potentieel probleem: Te weinig observaties? Te weinig sprekers? Relatief kleine dataset.

Best Trial:
Current best trial: 2c11dcfa with test_loss=0.24300648359691396 and parameters={'input': 13, 'output': 20, 'tunedir': PosixPath('/home/azureuser/code/ML22-tentamen/logs'), 'h1': 256, 'num_layers': 2}


Hypertuning: 22:11 09/02/2023 - Crashed due to low memory issue

Hoe bepaal je de grootte van de filters?
- Relatief kleine filters gebruiken omdat de datasets maar 13 features hebben.
- input channels: Gelijk aan het aantal features. In dit geval 13.
- In een linear model de hidden state niet te ver oprekken, kans op overfitting




Modellen om te proberen:
Gru + attention

Settings file omkatten
Model.py controleren
Scripts 01_model description controleren

In [ ]:
#Attention GRU template

class AttentionGRU(nn.Module):
    def __init__(
        self,
        config: Dict,
    ) -> None:
        super().__init__()
        self.rnn = nn.GRU(
            input_size=config["input_size"],
            hidden_size=config["hidden_size"],
            dropout=config["dropout"],
            batch_first=True,
            num_layers=config["num_layers"],
        )
        self.attention = nn.MultiheadAttention(
            embed_dim=config["hidden_size"],
            num_heads=4,
            dropout=config["dropout"],
            batch_first=True,
        )
        self.linear = nn.Linear(config["hidden_size"], config["output_size"])

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        x, _ = self.attention(x.clone(), x.clone(), x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [ ]:
#Werkende 01_model_design.py code
from datetime import datetime

import torch
from loguru import logger

from tentamen.data import datasets
from tentamen.model import Accuracy
from tentamen.settings import presets
from tentamen.train import trainloop

if __name__ == "__main__":
    logger.add(presets.logdir / "01.log")

    trainstreamer, teststreamer = datasets.get_arabic(presets)

    from tentamen.model import Linear
    from tentamen.settings import LinearConfig

    configs = [
        LinearConfig(
            input=13, output=20, tunedir=presets.logdir, h1=100, h2=10, dropout=0.5
        ),
    ]

    for config in configs:
        model = Linear(config.dict())  # type: ignore

        trainedmodel = trainloop(
            epochs=50,
            model=model,  # type: ignore
            optimizer=torch.optim.Adam,
            learning_rate=1e-3,
            loss_fn=torch.nn.CrossEntropyLoss(),
            metrics=[Accuracy()],
            train_dataloader=trainstreamer.stream(),
            test_dataloader=teststreamer.stream(),
            log_dir=presets.logdir,
            train_steps=len(trainstreamer),
            eval_steps=len(teststreamer),
        )

        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        path = presets.modeldir / (timestamp + presets.modelname)
        logger.info(f"save model to {path}")
        torch.save(trainedmodel, path)


Winnend model
Train c4bd491a_10
Dropout: 0.1905
H1= 256
Layers = 2


In [ ]:
#Werkend hypertuning
from typing import Dict

import ray
import torch
from filelock import FileLock
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB

from tentamen.data import datasets
from tentamen.model import Accuracy, Linear
from tentamen.settings import LinearSearchSpace, presets
from tentamen.train import trainloop


def train(config: Dict) -> None:
    datadir = presets.datadir

    with FileLock(datadir / ".lock"):
        trainstreamer, teststreamer = datasets.get_arabic(presets)

    model = Linear(config)  # type: ignore

    trainloop(
        epochs=30,
        model=model,  # type: ignore
        optimizer=torch.optim.Adam,
        learning_rate=1e-3,
        loss_fn=torch.nn.CrossEntropyLoss(),
        metrics=[Accuracy()],
        train_dataloader=trainstreamer.stream(),
        test_dataloader=teststreamer.stream(),
        log_dir=presets.logdir,
        train_steps=len(trainstreamer),
        eval_steps=len(teststreamer),
        tunewriter=True,
    )


if __name__ == "__main__":
    ray.init()

    config = LinearSearchSpace(
        input=13,
        output=20,
        tunedir=presets.logdir,
    )

    reporter = CLIReporter()
    reporter.add_metric_column("Accuracy")

    bohb_hyperband = HyperBandForBOHB(
        time_attr="training_iteration",
        max_t=30,
        reduction_factor=3,
        stop_last_trials=False,
    )

    bohb_search = TuneBOHB()

    analysis = tune.run(
        train,
        config=config.dict(),
        metric="test_loss",
        mode="min",
        progress_reporter=reporter,
        local_dir=config.tunedir,
        num_samples=20,
        search_alg=bohb_search,
        scheduler=bohb_hyperband,
        verbose=1,
    )

    ray.shutdown()
